# Reinforcement Learning - Thompson Sampling & the Multi-Armed Bandit Problem
This notebook, implements **Thompson Sampling** in order to solve an instance of the famous **Multi-Armed Bandit Problem**.

This notebook is adapted for the notebook in Dr. Daniel Soper's YouTube channel https://www.youtube.com/user/DanSoper33 and 
Nando de Freitas YouTube channel https://www.youtube.com/channel/UC0z_jCi0XWqI8awUuQRFnyw and Wikepedia (Thompson sampling https://en.wikipedia.org/wiki/Thompson_sampling and Multi-Armed Bandit Problem https://en.wikipedia.org/wiki/Multi-armed_bandit). In particular, the videos and Machine learning - Bayesian optimization and multi-armed bandits https://youtu.be/vz3D36VXefI.  The norebook also borrows from Thompson Sampling by Andre Cianflone  https://github.com/andrecianflone/thompson


### Thompson sampling  


**Thompson sampling**, named after William R. Thompson, is a
heuristic for choosing actions that addresses the
exploration-exploitation dilemma in the multi-armed bandit problem. It
consists of choosing the action that maximizes the expected reward with
respect to a randomly drawn belief.  The algorithm addresses a broad range of problems in a computationally efficient manner and is therefore enjoying wide use.  For a more extensive review of the theory, checkout A Tutorial on Thompson Sampling by Russo et al., 2017.  https://arxiv.org/abs/1707.02038



Thompson sampling is powerful but simple algorithm that implicitly balances exploration and exploitation based on quality and uncertainty. 
If we have a k-armed bandit and model the probability that each arm gives us a positive reward. The goal is of course to maximize our rewards by pulling the most promising arm. 

Thompson sampling, on the other hand, incorporates uncertainty by modelling the bandit's Bernouilli parameter with a prior beta distribution. Thompson sampling can be generalized to sample from any arbitrary distributions over its parameters.

The beauty of the algorithm is that it always chooses the action with the highest expected reward, with the twist that this reward is weighted by *uncertainty* through the posterior distribution.

This is a Bayesian approach to the bandit problem. In our Bernouilli bandit setup, each action $k$ returns reward of 1 with probability $\theta_k$, and 0 with probability $1-\theta_k$. At the beginning of a simulation, each $\theta_k$ is sampled from a uniform distribution $\theta_k \sim Uniform(0,1)$ with $\theta_k$ held constant for the rest of that simulation (in the stationary case). The agent begins with a prior belief of the reward of each arm $k$ with a beta distribution, where $\alpha = \beta = 1$. The prior probability density of each $\theta_k$ is:

$$
p(\theta_k) = \frac{\Gamma(\alpha_k + \beta_k)}{\Gamma(\alpha_k)\Gamma(\beta_k)} \theta_k^{\alpha_k -1} (1-\theta_k)^{\beta_k-1}
$$

An action is chosen by first sampling from the beta distribution, followed by choosing the action with highest mean reward:
$$
x_t = \text{argmax}_k (\hat{\theta}_k), \quad \hat{\theta}_k \sim \text{beta}(\alpha_k, \beta_k)
$$

According to Bayes' rule, an action's posterior distribution is updated depending on the reward $r_t$ received:
$$
(\alpha_k, \beta_k) = (\alpha_k, \beta_k) + (r_t, 1-r_t)
$$

Thus the actions' posterior distribution and uncertainty are constantly updated throughout the simulation.
The idea
-----------

The top graph below shows a posterior probability of a reward $P(x)$ (e.g. getting a winning pull on a slot machine or getting a click in an ad campaign), given some action space $x$ (e.g. choosing a slot machine or ad channel). The bands represent the uncertainty or variance around the estimates of $P(x)$. 

![Thompson sampling](https://raw.githubusercontent.com/nikbearbrown/Google_Colab/master/img/Thompson_Sampling.png)


_Thompson sampling from Nando de Freitas YouTube_

Note that the two points on the right dominate those on the left so we never will try those.  However we may try sampling at 0.8 (e.g. slot machine 8) as there is the possibility that when the uncertainty is reduced it is better than 0.9 (e.g. slot machine 8).  We choose points where $\mu({x})$ is high for exploitation. We choose points where $\mu({x})$ is high and $\sigma({x})$ is high for exploration. Note that in exploration there is no need to choose points where $\sigma({x})$ is zero during exploration as well are confident about those values.

An $\epsilon$-greedy algorithm would, with probability $\epsilon$, just as likely choose any action. The highly certain poor actions to the right of the graph are wasteful exploratory actiona.

In practice we want to balance this tradeoff of exploitation versus exploration with an acquistion function that takes the form:

$\mu({x}) + \kappa\sigma({x})$ 


 where $\kappa$ is a exporation parameter, a constant times the variance. If you think exporation is important then you would choose a large $\kappa$, a small $\kappa$, say $0.1$ would favor exploitation and a $\kappa$ of $0$ is pure exploitation.

In Thompson sampling one doesn't have an explcit $\kappa$ parameter, rather the uncertainty information explicitly captured via a posterior distribution. 

### Real-world example

Imagine you have \\$1,000 to play the slot machines (bandits) and it costs a dollar to play a bandit. There are six slot machines available, and each turn playing a machine costs. Or imagine you have \\$100,000 and are placing \\$100 bids in various social media advertising chanels. The probability of winning on any given turn (which is called the ***conversion rate***) is unknown. Note that in this simplified examples the slots always pay out the same amount, or in the advertising chanel a win is getting a click. How would you adjust the problem if the slots could pay out different amounts? 

The probility of winning varies from bandit to bandit, but is constant for that bandit. How would you adjust the problem if the bandit could change the payout in response the the environment? For example ad channels payout probility was dependent on what was being advertised. Some channels were better for some things.

The idea to to identify the best bandit or ad channel as quickly as possible.


### A simulation with slots

Let's look at some code. The notebook code is mostly from  Dr. Daniel Soper's YouTube channel https://www.youtube.com/user/DanSoper33 

In [ ]:
#import libraries
import numpy as np

### Define Environment

In [ ]:
#Define the total number of turns (i.e., the number of times we will play a slot machine).
#Remember, we have $1,000 available, and each turn costs $1. We thus have 1,000 turns.
number_of_turns = 1000

#define the total number of slot machines
number_of_bandits = 11

#Define arrays where we can keep track of our wins (positive rewards) 
#and losses (negative rewards) for each slot machine.
number_of_positive_rewards = np.zeros(number_of_bandits)
number_of_negative_rewards = np.zeros(number_of_bandits)

#define a seed for the random number generator (to ensure that results are reproducible)
np.random.seed(5)

#create a random conversion rate between 1% and 15% for each slot machine
# conversion_rates = np.random.uniform(0.001, 0.33, number_of_bandits)
conversion_rates = np.random.uniform(0.001, 0.33, number_of_bandits)

#Show conversion rates for each slot machine. Remember that in a real-world scenario
#the decision-maker would not know this information!
for i in range(number_of_bandits):
  print('Conversion rate for slot machine {0}: {1:.2%}'.format(i, conversion_rates[i]))

Conversion rate for slot machine 0: 7.40%
Conversion rate for slot machine 1: 28.75%
Conversion rate for slot machine 2: 6.90%
Conversion rate for slot machine 3: 30.32%
Conversion rate for slot machine 4: 16.17%
Conversion rate for slot machine 5: 20.23%
Conversion rate for slot machine 6: 25.30%
Conversion rate for slot machine 7: 17.16%
Conversion rate for slot machine 8: 9.86%
Conversion rate for slot machine 9: 6.28%
Conversion rate for slot machine 10: 2.76%


Note that if we get bandits/ad channels with very close high payout, say 13.19% and 13.86%. It may take a large number of tries to determine which is truely best. As the aquisition function simultainiously balances exploration and explotation then these two options should quickly $\mu({x}) + \kappa\sigma({x})$ dominate the rest so the learner will quickly make good choices.

Conversion rate for slot machine 0: 4.11%  
Conversion rate for slot machine 1: 13.19%  
Conversion rate for slot machine 2: 3.89%  
Conversion rate for slot machine 3: 13.86%  
Conversion rate for slot machine 4: 7.84%  
Conversion rate for slot machine 5: 9.56%  

### Create the Data Set

In [ ]:
#The data set is a matrix with one row for each turn, and one column for each slot machine.
#Each item in the matrix represents the outcome of what would happen if we were to play a  
#particular slot machine on that particular turn. A value of "1" indicates that we would win, 
#while a value of "0" indicates that we would lose. The number of "wins" for each slot machine
#is determined by its conversion rate.
outcomes = np.zeros((number_of_turns, number_of_bandits)) #create a two-dimensional numpy array, and fill it with zeros
for turn_index in range(number_of_turns): #for each turn
    for slot_machine_index in range(number_of_bandits): #for each slot machine
        #Get a random number between 0.0 and 1.0.
        #If the random number is less than or equal to this slot machine's conversion rate, then set the outcome to "1".
        #Otherwise, the outcome will be "0" because the entire matrix was initially filled with zeros.
        if np.random.rand() <= conversion_rates[slot_machine_index]:
            outcomes[turn_index][slot_machine_index] = 1

#display the first 15 rows of data
print(outcomes[0:15, 0:6]) #this sort of indexing means "rows 0 to 14" (i.e., the first 15 rows) and "columns 0 through 5" (i.e., the first six columns)

[[0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


One reads the row below as slot/ad channel 0 and 4 won and the rest lost.  

[1. 0. 0. 0. 1. 0.]

In [ ]:
#show means (i.e., conversion rates) for each column (i.e., for each slot machine)
for i in range(number_of_bandits):
  print('Mean for column {0}: {1:.2%}'.format(i, np.mean(outcomes[:, i])))

#show true conversion rate
for i in range(number_of_bandits):
  print('True conversion rate for column {0}: {1:.2%}'.format(i, conversion_rates[i]))  

Mean for column 0: 7.30%
Mean for column 1: 28.70%
Mean for column 2: 6.50%
Mean for column 3: 31.50%
Mean for column 4: 14.60%
Mean for column 5: 20.90%
Mean for column 6: 24.30%
Mean for column 7: 17.70%
Mean for column 8: 8.80%
Mean for column 9: 7.30%
Mean for column 10: 3.00%
True conversion rate for column 0: 7.40%
True conversion rate for column 1: 28.75%
True conversion rate for column 2: 6.90%
True conversion rate for column 3: 30.32%
True conversion rate for column 4: 16.17%
True conversion rate for column 5: 20.23%
True conversion rate for column 6: 25.30%
True conversion rate for column 7: 17.16%
True conversion rate for column 8: 9.86%
True conversion rate for column 9: 6.28%
True conversion rate for column 10: 2.76%


# Daniel Soper's Approach for K bandits with Thompson Sampling

Let's simulate using Thompson Sampling to determine which slot machine to play for each turn...

Note this simulation is using a beta distribution rather than a Guassianm becuase it is being used to model trails in the form of wins and losses.

In probability theory, the **beta distribution** is a
family of continuous probability distributions defined on the interval
\[0, 1\] parameterized by two positive shape parameters, denoted by
*α* and *β*, that appear as exponents of the random variable and control
the shape of the distribution.

The beta distribution has been applied to model the behavior of random
variables limited to intervals of finite length in a wide variety of
disciplines.

In Bayesian inference, the beta distribution is the [conjugate prior
probability distribution] for the Bernoulli, binomial, negative
binomial and geometric distributions. The beta distribution is a
suitable model for the random behavior of percentages and proportions.

![Beta distribution](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Beta_distribution_pdf.svg/1062px-Beta_distribution_pdf.svg.png)

_From https://en.wikipedia.org/wiki/Beta_distribution

In the simulation below note that the max_beta is the highest beta generated for that round.  So we don't have an explicit hyperparameter   $\kappa$ rather we are relying on taking a random sample. So rather than an acquistion function that takes the form:

$\mu({x}) + \kappa\sigma({x})$ 

it has this form

$\mathcal{X}$

where $\mathcal{X}$ is a random variable.

**Algorithm: Thompson($K$,$\alpha$, $\beta$)**  
for $t$ = 1,2, ..., do  
  &nbsp;&nbsp; // sample action parameter from beta distribution  
  &nbsp;&nbsp; for $k = 1, \dots, K$ do  
  &nbsp;&nbsp; &nbsp;&nbsp; Sample $\hat{\theta}_k \sim \text{beta}(\alpha_k, \beta_k)$  
  &nbsp;&nbsp; end for
  
  &nbsp;&nbsp; // select action, get reward  
  &nbsp;&nbsp; $x_t \leftarrow \text{argmax}_k \hat{\theta}_k$  
  &nbsp;&nbsp; $r_t \leftarrow \text{observe}(x_t)$  
  
  &nbsp;&nbsp; // update beta parameters  
  &nbsp;&nbsp; $(\alpha_{x_t}, \beta_{x_t}) \leftarrow (\alpha_{x_t}, \beta_{x_t})+(r_t, 1-r_t)$  
end for

The is known as beta-greedy 

In [ ]:
#for each turn

for turn_index in range(number_of_turns):
    index_of_machine_to_play = -1
    max_beta = -1 # note that max beta

    #determine which slot machine to play for this turn
    for slot_machine_index in range(number_of_bandits): #for each slot machine
        #Define the shape parameters for the beta distribution. The shape will depend on the number
        #of wins and losses that have thus far been observed for this particular slot machine.
        a = number_of_positive_rewards[slot_machine_index] + 1
        b = number_of_negative_rewards[slot_machine_index] + 1

        #Get a random value from the beta distribution whose shape is defined by the number of
        #wins and losses that have thus far been observed for this slot machine
        random_beta = np.random.beta(a, b)

        #if this is the largest beta value thus far observed for this iteration
        if random_beta > max_beta:
            max_beta = random_beta #update the maximum beta value thus far observed
            index_of_machine_to_play = slot_machine_index #set the machine to play to the current machine
    
    #play the selected slot machine, and record whether we win or lose
    if outcomes[turn_index][index_of_machine_to_play] == 1:
        number_of_positive_rewards[index_of_machine_to_play] += 1
    else:
        number_of_negative_rewards[index_of_machine_to_play] += 1

print('Number of turns {0}:'.format(number_of_turns))  

#compute and display the total number of times each slot machine was played
number_of_times_played = number_of_positive_rewards + number_of_negative_rewards 
for slot_machine_index in range(number_of_bandits): #for each slot machine
    print('Slot machine {0} was played {1} times that is, {2:.2%}'.format(slot_machine_index, number_of_times_played[slot_machine_index], (number_of_times_played[slot_machine_index]/number_of_turns)))

#identify and display the best slot machine to play
print('\nOverall Conclusion: The best slot machine to play is machine {}!'.format(np.argmax(number_of_times_played)))

#show true conversion rate
for i in range(6):
  print('True conversion rate for column {0}: {1:.2%}'.format(i, conversion_rates[i]))  

Number of turns 1000:
Slot machine 0 was played 13.0 times that is, 1.30%
Slot machine 1 was played 478.0 times that is, 47.80%
Slot machine 2 was played 39.0 times that is, 3.90%
Slot machine 3 was played 72.0 times that is, 7.20%
Slot machine 4 was played 44.0 times that is, 4.40%
Slot machine 5 was played 68.0 times that is, 6.80%
Slot machine 6 was played 154.0 times that is, 15.40%
Slot machine 7 was played 67.0 times that is, 6.70%
Slot machine 8 was played 35.0 times that is, 3.50%
Slot machine 9 was played 16.0 times that is, 1.60%
Slot machine 10 was played 14.0 times that is, 1.40%

Overall Conclusion: The best slot machine to play is machine 1!
True conversion rate for column 0: 7.40%
True conversion rate for column 1: 28.75%
True conversion rate for column 2: 6.90%
True conversion rate for column 3: 30.32%
True conversion rate for column 4: 16.17%
True conversion rate for column 5: 20.23%


### Compare the Performance of Thompson Sampling vs. a Random Sampling Strategy

Note that a random sampling can be a reasonable baseline to compare many RL algorithms.

In [ ]:
#compute total number of wins using Thompson Sampling strategy
total_wins_thompson_sampling = np.sum(number_of_positive_rewards)

#determine how many times we would win if we randomly choose a slot machine to play for each turn
total_wins_random_sampling = 0
for turn_index in range(number_of_turns):
  index_of_machine_to_play = np.random.randint(0, number_of_bandits) #randomly choose a machine to play
  if outcomes[turn_index][index_of_machine_to_play] == 1:
    total_wins_random_sampling += 1

#display results
print('Total wins with Thompson Sampling: {0:.0f}'.format(total_wins_thompson_sampling))
print('Total wins with Random Sampling: {0:.0f}'.format(total_wins_random_sampling))

Total wins with Thompson Sampling: 237
Total wins with Random Sampling: 163


# Andre Cianflone's Approach for K bandits with Thompson Sampling

Andre Cianflone also implements the two beta algorithm form and compares it to t the $\epsilon$-greedy algorithm and Upper Confidence Bound (UBC) algorithm. See https://github.com/andrecianflone/thompson
and https://colab.research.google.com/drive/1BHVH712x2Q2As9E5nN5Y8UR74T8w6AMO#scrollTo=lnzzQ4WZFLQ1





## References
[1]: Russo, Daniel, Benjamin Van Roy, Abbas Kazerouni, and Ian Osband. "A Tutorial on Thompson Sampling." arXiv preprint arXiv:1707.02038 (2017). [link](https://arxiv.org/abs/1707.02038)

[2]: Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. Vol. 1, no. 1. Cambridge: MIT press, 1998.